In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns 
import statsmodels.stats.proportion as proportion
from scipy.stats import ttest_ind,mannwhitneyu,shapiro,norm
from statsmodels.stats.weightstats import ztest
from tqdm import tqdm
import timeit
from scipy import stats
import math
from datetime import date, datetime, timedelta
import time
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import warnings
warnings.filterwarnings("ignore")

from credential import postgres_secret,clickhouse_dwh_secret

def get_engine(user):
    if user == postgres_secret['user']:
        db_name = postgres_secret['db_name']
        password = postgres_secret['password']
        host = postgres_secret['host']
        engine = create_engine(f'postgresql://{user}:{password}@{host}:6432/{db_name}')
    elif user == clickhouse_dwh_secret['user']:
            db_name = clickhouse_dwh_secret['db_name'] 
            password = clickhouse_dwh_secret['password']
            host = clickhouse_dwh_secret['host']
            engine = create_engine(f'clickhouse://{user}:{password}@{host}:8123/{db_name}')
    return engine
    

    
def execute(SQL, user):
    start_time = time.time()  # запоминаем время начала выполнения функции
    engine = get_engine(user)
    Session = sessionmaker(bind=engine)  # sessions factory ()
    with Session() as session: # open session
        result = session.execute(text(SQL))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        
    end_time = time.time()  # запоминаем время окончания выполнения функции
    execution_time = round(end_time - start_time,4) # вычисляем время выполнения   
    
    print(f"Время выполнения функции: {execution_time} секунд")
    print()
    return df

In [ ]:
df = pd.read_csv('/Users/kemran/Downloads/data.csv',sep=',')

In [ ]:
(df['dateAdded'].head())

In [ ]:
df.head()

In [ ]:
df[df['city']=='California'].index

In [ ]:
# Фильтруем DataFrame и получаем индексы  
california_indexes = df[(df['city'] == 'California') & (df['name']=='Taco Bell')].index.tolist()  

# Выводим индексы в виде списка  
print(california_indexes)

In [ ]:
df[((df['city'] == 'New York') | (df['name']=='Taco Bell')) & (~df['menus.name'].isin(['Volcano Taco','Fresco Soft Taco']))]


In [ ]:
type(df[['categories']])

In [ ]:
df.groupby(by = 'city',as_index=False).size()#.sort_values(by='id',ascending=False).head()['city'].tolist()

In [ ]:
df.groupby(by = 'city')['id'].count().reset_index()

In [ ]:
df['dateAdded'] = pd.to_datetime(df['dateAdded'])

df[df['dateAdded'].dt.month==10].sort_values(by = 'dateAdded').index

In [ ]:
import pandas as pd  


df['dateAdded'] = pd.to_datetime(df['dateAdded'])  
result = df.groupby(df['dateAdded'].dt.month)['id'].nunique().reset_index()

result = result.set_index('dateAdded')

result

In [ ]:
import pandas as pd  

# Убедимся, что даты имеют правильный формат  
df['dateUpdated'] = pd.to_datetime(df['dateUpdated'])  
df['dateAdded'] = pd.to_datetime(df['dateAdded'])  

# Добавляем столбец update_delta, рассчитывающий разницу в днях  
df['update_delta'] = (df['dateUpdated'] - df['dateAdded']).dt.days  

# Группируем данные по городам и рассчитываем необходимые значения  
result = df.groupby('city').agg(  
    update_delta_mean=('update_delta', 'mean'),  
    northern_latitude=('latitude', 'max')  # Предполагаем, что широта хранится в столбце 'latitude'  
).reset_index()  

# Вычисляем среднее значение update_delta для города Zephyrhills  
zep_mean = result.loc[result['city'] == 'Zephyrhills', 'update_delta_mean'].values[0]  

# Вывод результата  
print(result)  
print("Среднее значение update_delta для Zephyrhills:", zep_mean)  

In [ ]:
import pandas as pd  

# Убедимся, что даты имеют правильный формат  
df['dateUpdated'] = pd.to_datetime(df['dateUpdated'])  
df['dateAdded'] = pd.to_datetime(df['dateAdded'])  

# Добавляем столбец update_delta, рассчитывающий разницу в днях  
df['update_delta'] = (df['dateUpdated'] - df['dateAdded']).dt.days  


result = df.groupby('city').agg(  
    update_delta=('update_delta', 'mean'),  
    northern_latitude=('latitude', 'max')  # Предполагаем, что широта хранится в столбце 'latitude'  
).reset_index()  

result['zap_mean'] = result[result['city']=='Zephyrhills']['update_delta'].mean()

result

In [ ]:
import pandas as pd  

# Убедимся, что даты имеют правильный формат  
data['dateUpdated'] = pd.to_datetime(data['dateUpdated'])  
data['dateAdded'] = pd.to_datetime(data['dateAdded'])  

# Добавляем столбец update_delta, рассчитывающий разницу в днях  
data['update_delta'] = (data['dateUpdated'] - data['dateAdded']).dt.days  


result = data.groupby('city').agg(  
    update_delta=('update_delta', 'mean'),  
    northern_latitude=('latitude', 'max')  # Предполагаем, что широта хранится в столбце 'latitude'  
).reset_index()  

result['zap_mean'] = result[result['city']=='Zephyrhills']['update_delta'].mean()

result

In [ ]:
import pandas as pd  

# Пример создания DataFrame (замените на ваш DataFrame data)  
# data = pd.DataFrame({ ... })  

# Фильтрация ресторанов, где в categories упоминается 'Pizza'  
pizza_restaurants = df[df['categories'].str.contains('Pizza', na=False)]  

# Получение индекса 3-го элемента после фильтрации  
third_element_index = pizza_restaurants.index[2]  # Индексы начинаются с 0, поэтому 2 - это 3-й элемент  

# Вывод индекса третьего элемента  
print(third_element_index)

In [ ]:
import pandas as pd  

# Сначала мы предполагаем, что DataFrame data уже существует и содержит нужные колонки  
# Пример: data = pd.DataFrame({ ... })   

# Преобразование дат в формат datetime  
df['dateUpdated'] = pd.to_datetime(df['dateUpdated'])  
df['dateAdded'] = pd.to_datetime(df['dateAdded'])  

# Добавляем столбец update_delta, рассчитывающий разницу в днях  
df['update_delta'] = (df['dateUpdated'] - df['dateAdded']).dt.days  

# Рассчитываем среднее и медиану для столбца update_delta  
mean_update_delta = df['update_delta'].mean()  
median_update_delta = df['update_delta'].median()  

# Округляем до 2 знаков после запятой  
mean_update_delta = round(mean_update_delta, 2)  
median_update_delta = round(median_update_delta, 2)  

# Выводим результаты через пробел  
print(f"{mean_update_delta} {median_update_delta}")

In [ ]:
import pandas as pd  

# Пример создания DataFrame (замените на ваш DataFrame data)  
# data = pd.DataFrame({ ... })  

# Предполагаем, что categories - это строка, где категории разделены запятыми  
# Фильтруем записи с более чем 20 категориями  
df['category_count'] = df['categories'].str.split(',').str.len()  # Предполагаем, что категории разделены запятыми  
filtered_data = df[df['category_count'] > 20]  

# Группируем по провинциям и вычисляем минимальную longitude  
result = filtered_data.groupby('province')['longitude'].min().reset_index()  

# Округляем значения longitude до 3-х знаков после запятой  
result['longitude'] = result['longitude'].round(3)  

# Сохранение в CSV с сепаратором ;  
result.to_csv('filtered_data.csv', sep=';', index=False)  

print("Файл успешно сохранен как 'filtered_df.csv'")  

In [ ]:
df['dateAdded'].head(3)

# Мини проект 

In [ ]:
df = pd.read_csv('/Users/kemran/Downloads/playstore.csv',sep=',')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['App'].nunique()

In [ ]:
df['Rating'].isna().sum()

In [ ]:
slice1 = df.iloc[0:3]
slice2 = df.iloc[5:8]
slice3 = df.iloc[15:19]

df_concat = pd.concat([slice1,slice2,slice3])
result_df = df_concat[['App', 'Size', 'Genres', 'Current Ver']]
result_df.to_csv('result.csv')

In [ ]:
t = df.drop_duplicates(subset='App').reset_index(drop=True)

In [ ]:
t_agg = t.groupby('Type')['App'].nunique().reset_index()

t_agg['frac'] = t_agg['App'] / t_agg['App'].sum()

t_agg

In [ ]:
d = df.drop_duplicates(subset='App').reset_index(drop=True)
d.columns = d.columns.str.lower().str.replace(' ', '_')

d[(d['category']=='EDUCATION') & (d['reviews']>=1000)].reset_index(drop=True)

In [ ]:
df.drop_duplicates(subset='App').reset_index(drop=True)

In [ ]:
d = df.drop_duplicates(subset='App').reset_index(drop=True)
d.columns = d.columns.str.lower().str.replace(' ', '_')

d[(d['category']=='EDUCATION') & (d['reviews']>=1000)].reset_index(drop=True)

d['price'] = d['price'].str.replace('$', '').reset_index().astype('float')['price'].reset_index(drop=True)


df_pivot = pd.pivot_table(data = d,
                          index = ['category','type'],
                          values = ['price','rating','reviews'],
                          aggfunc = 'mean')

df_pivot.rename(columns = {'price':'mean_price',
                           'rating':'mean_rating',
                           'reviews':'mean_reviews'},
                inplace=True)

df_pivot['mean_price'] = df_pivot['mean_price'].round(2)
df_pivot['mean_reviews'] = df_pivot['mean_reviews'].round(2)
df_pivot['mean_rating'] = df_pivot['mean_rating'].round(1)
df_pivot.to_csv('final.csv',sep=',')

In [ ]:
t = d.groupby(['category','type'])[['price','rating','reviews']].mean()

In [ ]:
type(t)

In [ ]:
df_pivot = pd.pivot_table(data = d,
                          index = ['category','type'],
                          values = ['price','rating','reviews'],
                          aggfunc = 'mean')

In [ ]:
type(df_pivot)

# SQL

In [2]:
database="startml",
user="robot-startml-ro",
password="pheiph0hahj1Vaif",
host="postgres.lab.karpov.courses",
port=6432

import pandas as pd

feed_action = pd.read_sql(
    """SELECT * FROM "feed_action"  WHERE action='view' AND time::date>='2021-12-01' LIMIT 100 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

feed_action.head(100)

,user_id,post_id,action,time
0,144732,3003,view,2021-12-01 00:00:00
1,108396,3246,view,2021-12-01 00:00:00
2,5215,3192,view,2021-12-01 00:00:01
3,150000,2997,view,2021-12-01 00:00:01
4,144732,3166,view,2021-12-01 00:00:01
...,...,...,...,...
95,51127,3246,view,2021-12-01 00:00:17
96,124979,3199,view,2021-12-01 00:00:17
97,147885,3177,view,2021-12-01 00:00:18
98,50516,3071,view,2021-12-01 00:00:18


In [4]:
user = pd.read_sql(
    """SELECT * FROM "user"  """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [6]:
post = pd.read_sql(
    """SELECT *
        FROM "post"
        
        """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

post.head()

,id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [ ]:
post['len'] = post['text'].str.len()

post.groupby('topic')['len'].max()

In [ ]:
user[(user['age']>=30) & (user['os']=='iOS')]

In [ ]:
pd.read_csv('/Users/kemran/Downloads/sql (1).csv')

In [ ]:
user = pd.read_sql(
    """ SELECT exp_group,city,avg(age) AS mean_age, count(id) AS cnt_user
        FROM "user"
        WHERE city='Omsk'
        GROUP BY 1,2
        --HAVING avg(age)>=27.2
        
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user.head(10)

#user.to_csv('sql.csv',sep=',',index=False)

## Для финального проекта

In [ ]:
user = pd.read_sql(
    """ SELECT t2.post_id,count(t1.id) AS cnt_like,count(distinct t1.id) as cnt_user,max(t2.time) as max_time
        FROM (SELECT * FROM "user") t1
        INNER JOIN (SELECT * FROM "feed_action" WHERE action='like') t2 on t1.id=t2.user_id
        INNER JOIN (SELECT * FROM "post" WHERE topic='covid') t3 on t1.post_id=t2.id
        GROUP BY 1
        ORDER BY cnt_like DESC
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user.head(100)

#user.to_csv('sql.csv',sep=',',index=False)

In [16]:
user = pd.read_sql(
    """ SELECT 
        t1.post_id,
        count(t1.user_id) AS cnt_like,
        count(distinct t1.user_id) as cnt_user,
        max(t1.time) as max_time
        FROM (SELECT * FROM "feed_action" WHERE action='like') AS t1
        INNER JOIN (SELECT * FROM "post") AS  t2 on t1.post_id=t2.id
        GROUP BY 1
        ORDER BY 2 DESC 
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user.head(100)

#user.to_csv('sql.csv',sep=',',index=False)

,post_id,cnt_like,cnt_user,max_time
0,4866,31714,31714,2022-01-01 13:57:43
1,5127,29637,29637,2022-01-05 13:52:17
2,6448,27939,27939,2022-01-22 05:50:39
3,6498,27729,27729,2022-01-22 21:45:11
4,6513,27686,27686,2022-01-22 19:59:52
...,...,...,...,...
95,6281,14563,14563,2022-01-18 23:57:42
96,5659,14533,14533,2022-01-09 21:58:42
97,6737,14526,14526,2022-01-23 19:57:01
98,5166,14463,14463,2022-01-03 21:57:11


In [ ]:
user = pd.read_sql(
    """ SELECT *
        FROM "feed_action"
        WHERE action='like'
        LIMIT 100

    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user.head(100)

In [ ]:
user.to_csv('sql2.csv',index=False)

In [ ]:
pd.read_csv('/Users/kemran/Downloads/sql2 (2).csv')